In [1]:
import os
# Create a Kaggle directory in your Colab environment
os.makedirs('/root/.kaggle', exist_ok=True)
# Move your kaggle.json file to the .kaggle folder
!mv kaggle.json /root/.kaggle/
# Set permissions for the file
!chmod 600 /root/.kaggle/kaggle.json


In [2]:
!pip install kagglehub


In [3]:
import kagglehub

# Download the latest version of the dataset
path = kagglehub.dataset_download("arifmia/football-player-image-dataset")

print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1


In [6]:
import os

# List the contents of the dataset directory
print("Dataset Files:")
for root, dirs, files in os.walk(path):
    print(root, "contains", len(files), "files")


Dataset Files:
/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1 contains 0 files
/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1/footbll  player contains 0 files
/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1/footbll  player/messi contains 121 files
/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1/footbll  player/ronalu contains 133 files
/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1/footbll  player/devala contains 131 files
/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1/footbll  player/dimaria contains 130 files
/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1/footbll  player/neimar contains 127 files
/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1/footbll  player/matiaz contains 124 files


In [7]:
import os

# Define the dataset path
dataset_path = "/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1/footbll player"


In [4]:
# Install required libraries
!pip install facenet-pytorch
!pip install opencv-python
!pip install numpy
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166

In [4]:
import torch
import numpy as np
import cv2
import os
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image

In [6]:
# Updated dataset path
dataset_path = '/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1/footbll player'
output_path = '/content/processed_images'


# Create output directory
os.makedirs(output_path, exist_ok=True)

In [7]:
# Initialize models
mtcnn = MTCNN(keep_all=True, device='cuda' if torch.cuda.is_available() else 'cpu')
resnet = InceptionResnetV1(pretrained='vggface2').eval()

def process_images_and_generate_embeddings(dataset_path, output_path):
    player_embeddings = {}

    # Iterate through player folders
    for player_folder in os.listdir(dataset_path):
        player_path = os.path.join(dataset_path, player_folder)

        if os.path.isdir(player_path):
            # Create output folder for each player
            player_output = os.path.join(output_path, player_folder)
            os.makedirs(player_output, exist_ok=True)

            player_faces = []

            # Process each image in the player's folder
            for image_file in os.listdir(player_path):
                try:
                    # Full path to the image
                    img_path = os.path.join(player_path, image_file)

                    # Open image
                    img = Image.open(img_path)

                    # Detect faces
                    faces, probs = mtcnn(img, return_prob=True)

                    if faces is not None:
                        # Generate embeddings
                        with torch.no_grad():
                            embeddings = resnet(faces)

                        # Store embeddings
                        for embedding in embeddings:
                            player_faces.append(embedding.numpy())

                        # Save detected faces
                        for i, face in enumerate(faces):
                            face_img = transforms.ToPILImage()(face)
                            face_img.save(os.path.join(player_output, f'{image_file}_face_{i}.jpg'))

                except Exception as e:
                    print(f"Error processing {image_file} for {player_folder}: {e}")

            # Average embeddings for the player
            if player_faces:
                player_embeddings[player_folder] = np.mean(player_faces, axis=0)

    return player_embeddings

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [9]:
import os

# Print the full path details
print(os.path.exists('/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1'))

# List the contents of the directory
print(os.listdir('/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1'))

True
['footbll  player']


In [97]:
import torch
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import torchvision.transforms as transforms

# Configuration
base_path = '/root/.cache/kagglehub/datasets/arifmia/football-player-image-dataset/versions/1/footbll  player'
output_path = '/content/processed_images'

# Create output directory
os.makedirs(output_path, exist_ok=True)

# Advanced MTCNN Configuration
mtcnn = MTCNN(
    image_size=160,       # Consistent image size
    margin=0,             # No margin around detected face
    min_face_size=20,     # Minimum face size to detect
    thresholds=[0.6, 0.7, 0.7],  # Relaxed detection thresholds
    factor=0.709,         # Scale factor for multi-scale detection
    keep_all=True,        # Keep all detected faces
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# Initialize face recognition model
resnet = InceptionResnetV1(pretrained='vggface2').eval()

def preprocess_image(img):
    """Advanced image preprocessing for consistent input"""
    transform = transforms.Compose([
        transforms.Resize((160, 160)),  # Consistent resize
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
    ])
    return transform(img)

def calculate_face_similarity(known_embedding, face_embedding):
    """Calculate cosine similarity between face embeddings"""
    cosine_similarity = 1 - torch.nn.functional.cosine_similarity(
        torch.tensor(known_embedding).unsqueeze(0),
        torch.tensor(face_embedding).unsqueeze(0)
    ).item()
    return cosine_similarity

def adaptive_threshold(embeddings):
    """Dynamically calculate recognition threshold"""
    distances = [np.linalg.norm(emb) for emb in embeddings]
    mean_distance = np.mean(distances)
    std_distance = np.std(distances)
    return mean_distance + 0.5 * std_distance

def process_images_and_generate_embeddings(base_path, output_path):
    player_embeddings = {}

    # List all player folders
    player_folders = os.listdir(base_path)

    print("Players found:", player_folders)

    for player_folder in player_folders:
        player_path = os.path.join(base_path, player_folder)

        if os.path.isdir(player_path):
            # Create output folder for each player
            player_output = os.path.join(output_path, player_folder)
            os.makedirs(player_output, exist_ok=True)

            player_faces = []

            # Process each image in the player's folder
            for image_file in os.listdir(player_path):
                try:
                    # Full path to the image
                    img_path = os.path.join(player_path, image_file)

                    # Check if it's an image file
                    if not image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                        print(f"Skipping non-image file: {image_file}")
                        continue

                    # Open image
                    img = Image.open(img_path)

                    # Detect faces
                    faces, probs = mtcnn(img, return_prob=True)

                    if faces is not None and len(faces) > 0:
                        print(f"Detected {len(faces)} faces for {player_folder}")

                        # Generate embeddings with high confidence faces
                        high_conf_faces = [face for face, prob in zip(faces, probs) if prob > 0.95]

                        if high_conf_faces:
                            with torch.no_grad():
                                # Stack the faces into a batch tensor
                                face_tensor = torch.stack(high_conf_faces)

                                # Get embeddings for the batch of faces
                                embeddings = resnet(face_tensor)

                            # Store embeddings for the player
                            for embedding in embeddings:
                                player_faces.append(embedding.numpy())

                        # Save detected faces
                        for i, face in enumerate(high_conf_faces):
                            face_img = transforms.ToPILImage()(face)
                            face_img.save(os.path.join(player_output, f'{image_file}_face_{i}.jpg'))

                except Exception as e:
                    print(f"Error processing {image_file} for {player_folder}: {e}")

            # Average embeddings for the player
            if player_faces:
                player_embeddings[player_folder] = np.mean(player_faces, axis=0)
                print(f"Processed {player_folder}: {len(player_faces)} face embeddings")
            else:
                print(f"No faces detected for {player_folder}")

    return player_embeddings

# Generate embeddings
player_embeddings = process_images_and_generate_embeddings(base_path, output_path)

# Save embeddings for later use
import pickle
with open('/content/player_embeddings.pkl', 'wb') as f:
    pickle.dump(player_embeddings, f)

print("\nEmbeddings saved to /content/player_embeddings.pkl")


Players found: ['messi', 'ronalu', 'devala', 'dimaria', 'neimar', 'matiaz']
Detected 1 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 7 faces for messi
Detected 3 faces for messi
Detected 3 faces for messi
Detected 2 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 2 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 2 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 2 faces for messi
Detected 2 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 2 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 2 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 1 faces for messi
Detected 2 faces for messi
Detected 1 faces for messi
Detect

In [120]:
import os
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
import torchvision.transforms as transforms
import pickle

# Load pre-generated embeddings
with open('/content/player_embeddings.pkl', 'rb') as f:
    player_embeddings = pickle.load(f)

# Advanced MTCNN Configuration
mtcnn = MTCNN(
    image_size=160,       # Consistent image size
    margin=0,             # No margin around detected face
    min_face_size=20,     # Minimum face size to detect
    thresholds=[0.6, 0.7, 0.7],  # Relaxed detection thresholds
    factor=0.709,         # Scale factor for multi-scale detection
    keep_all=True,        # Keep all detected faces
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# Initialize face recognition model
resnet = InceptionResnetV1(pretrained='vggface2').eval()

def calculate_face_similarity(known_embedding, face_embedding):
    """Calculate cosine similarity between face embeddings"""
    cosine_similarity = 1 - torch.nn.functional.cosine_similarity(
        torch.tensor(known_embedding).unsqueeze(0),
        torch.tensor(face_embedding).unsqueeze(0)
    ).item()
    return cosine_similarity

def mark_attendance(input_image, player_embeddings, threshold=0.6):
    """Mark attendance with clear player name display"""
    # Load input image
    img = Image.open(input_image)

    # Detect faces
    faces, probs = mtcnn(img, return_prob=True)

    # Initialize attendance results with all players marked absent
    attendance_results = {player: False for player in player_embeddings.keys()}

    if faces is not None and len(faces) > 0:
        # Generate embeddings for detected faces
        with torch.no_grad():
            input_embeddings = resnet(faces)

        # Analyze each detected face
        for i, (face, prob) in enumerate(zip(faces, probs)):
            # Compute similarities to all known players
            similarities = {}
            for player, known_embedding in player_embeddings.items():
                similarity = calculate_face_similarity(known_embedding, input_embeddings[i])
                similarities[player] = similarity

            # Find best match
            best_match = min(similarities, key=similarities.get)
            best_similarity = similarities[best_match]

            # Mark attendance if similarity is below threshold
            if best_similarity < threshold:
                attendance_results[best_match] = True

    # Print clear attendance results
    print("\nAttendance Results:")
    for player in sorted(attendance_results.keys()):
        status = "Present ✓" if attendance_results[player] else "Absent ✗"
        print(f"{player}: {status}")

    return attendance_results

def update_attendance_log(attendance_results, log_path='/content/attendance_log.csv'):
    """Update attendance log with improved error handling"""
    try:
        # Get current date and time
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Convert boolean attendance to 'Present'/'Absent'
        attendance_results = {player: "Present" if present else "Absent" for player, present in attendance_results.items()}

        # Prepare attendance record
        record = pd.DataFrame({
            'Timestamp': [current_time],
            **attendance_results
        })

        # Check if log exists
        if os.path.exists(log_path):
            # Read existing log
            df = pd.read_csv(log_path)
            # Concatenate new record
            df = pd.concat([df, record], ignore_index=True)
        else:
            # Use new record as initial DataFrame
            df = record

        # Save updated log
        df.to_csv(log_path, index=False)

        print("\nAttendance Log Updated Successfully!")
        return df

    except Exception as e:
        print(f"Error updating attendance log: {e}")
        return None

def attendance_workflow(test_image_path, threshold=0.6):
    """Comprehensive attendance workflow"""
    try:
        # Mark attendance
        attendance = mark_attendance(test_image_path, player_embeddings, threshold)

        # Update attendance log
        log = update_attendance_log(attendance)

        return attendance, log

    except Exception as e:
        print(f"Error in attendance workflow: {e}")
        return None, None

# Diagnostic Information
print("\nAttendance Tracking System is Ready!")
print("Players in System:", list(player_embeddings.keys()))
print("\nTo mark attendance:")
print("1. Prepare a test image with players")
print("2. Use attendance_workflow(image_path)")



Attendance Tracking System is Ready!
Players in System: ['messi', 'ronalu', 'devala', 'dimaria', 'neimar', 'matiaz']

To mark attendance:
1. Prepare a test image with players
2. Use attendance_workflow(image_path)


In [123]:
# Replace with your actual image path
test_image_path = '/content/neymar.jpg'
attendance, log = attendance_workflow(test_image_path)




Attendance Results:
devala: Absent ✗
dimaria: Absent ✗
matiaz: Absent ✗
messi: Absent ✗
neimar: Present ✓
ronalu: Absent ✗

Attendance Log Updated Successfully!


<ipython-input-120-d2fa666a54a3>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(face_embedding).unsqueeze(0)
